### Image Restoration of President Lincoln Portrait. 

In [1]:
# https://www.washingtonpost.com/entertainment/museums/photographing-a-nation-in-motion/2015/09/17/64f77616-5d55-11e5-8e9e-dce8a2a2a679_story.html

<html>
<img src="images/Lincoln2.jpg", width = 300 height = 600>
<html>    

In [2]:
import numpy as np
import cv2 as cv

In [3]:
# OpenCV Utility Class for Mouse Handling
class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        self.show()
        cv.setMouseCallback(self.windowname, self.on_mouse)

    def show(self):
        cv.imshow(self.windowname, self.dests[0])
        cv.imshow(self.windowname + ": mask", self.dests[1])

    # onMouse function for Mouse Handling
    def on_mouse(self, event, x, y, flags, param):
        pt = (x, y)
        if event == cv.EVENT_LBUTTONDOWN:
            self.prev_pt = pt
        elif event == cv.EVENT_LBUTTONUP:
            self.prev_pt = None

        if self.prev_pt and flags & cv.EVENT_FLAG_LBUTTON:
            for dst, color in zip(self.dests, self.colors_func()):
                cv.line(dst, self.prev_pt, pt, color, 7)
            self.dirty = True
            self.prev_pt = pt
            self.show()

In [4]:
# Read image in color mode
filename = "C:/Users/alket/OneDrive/Desktop/MachineLearningCV/images/110.jpg"
img = cv.imread(filename, cv.IMREAD_COLOR)

# If image is not read properly, return error
if img is None:
    print('Failed to load image file: {}'.format(filename))

In [5]:
print(img)

[[[255 251 252]
  [255 251 252]
  [255 251 252]
  ...
  [ 70  93 115]
  [ 70  93 115]
  [ 70  93 115]]

 [[255 251 252]
  [255 251 252]
  [255 251 252]
  ...
  [ 67  90 112]
  [ 67  90 112]
  [ 67  90 112]]

 [[255 251 252]
  [255 251 252]
  [255 251 252]
  ...
  [ 48  71  93]
  [ 48  71  93]
  [ 48  71  93]]

 ...

 [[ 65  83 106]
  [ 69  87 110]
  [ 82 100 123]
  ...
  [ 76  90 112]
  [ 76  91 110]
  [ 76  91 110]]

 [[ 57  75  98]
  [ 63  81 104]
  [ 78  96 119]
  ...
  [ 76  90 112]
  [ 76  91 110]
  [ 76  91 110]]

 [[ 56  74  97]
  [ 61  79 102]
  [ 75  93 116]
  ...
  [ 75  89 111]
  [ 75  90 109]
  [ 75  90 109]]]


In [6]:
# Create a copy of original image
img_mask = img.copy()
# Create a black copy of original image
# Acts as a mask
inpaintMask = np.zeros(img.shape[:2], np.uint8)
# Create sketch using OpenCV Utility Class: Sketcher
sketch = Sketcher('image', [img_mask, inpaintMask], lambda : ((255, 255, 255), 255))

In [ ]:
while True:
    ch = cv.waitKey()
    if ch == 27:
        break
    if ch == ord('t'):
        # Use Algorithm proposed by Alexendra Telea: Fast Marching Method (2004)
        # Reference: https://pdfs.semanticscholar.org/622d/5f432e515da69f8f220fb92b17c8426d0427.pdf
        res = cv.inpaint(src=img_mask, inpaintMask=inpaintMask, inpaintRadius=3, flags=cv.INPAINT_TELEA)
        cv.imshow('Inpaint Output using FMM', res)
    if ch == ord('n'):
        # Use Algorithm proposed by Bertalmio, Marcelo, Andrea L. Bertozzi, and Guillermo Sapiro: 
        # Navier-Stokes, Fluid Dynamics, and Image and Video Inpainting (2001)
        res = cv.inpaint(src=img_mask, inpaintMask=inpaintMask, inpaintRadius=3, flags=cv.INPAINT_NS)
        cv.imshow('Inpaint Output using NS Technique', res)
    if ch == ord('r'):
        img_mask[:] = img
        inpaintMask[:] = 0
        sketch.show()

In [ ]:
cv.destroyAllWindows()